In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import OrderedDict
from random import randint
import random
import decimal
from operator import add
from operator import itemgetter

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_hidden_layers = 9
max_neurons_per_layer = 100
min_learning_rate = .0001
max_learning_rate = .01
acti_func = ["softmax", "elu", "selu", "softplus", "softsign", 
             "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
num_individuals = 64
loss_weight = 10
accuracy_weight = 10
time_weight = 10

In [3]:
def load_data():
    iris_data = load_iris()
    print('Example Data:')
    print(iris_data.data[:5])
    print('Example Labels:')
    print(iris_data.target[:5])
    global iris_data

<ipython-input-3-11c664384486>:7: SyntaxWarning: name 'iris_data' is assigned to before global declaration
  global iris_data


In [4]:
def process_data():
    x = iris_data.data
    y_ = iris_data.target.reshape(-1,1)

    encoder = OneHotEncoder(sparse=False)
    y = encoder.fit_transform(y_)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)
    global train_x, test_x, train_y, test_y

<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y


In [5]:
load_data()
process_data()

Example Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example Labels:
[0 0 0 0 0]


In [6]:
def create_network(model_num):
    model_name = "Model"+str(model_num)
    model_desc = {'name':model_name}
    print(model_name)
    model_name = Sequential()
    num_hidden_layers = randint(0, max_hidden_layers)
    i = 0
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    while i < num_hidden_layers:
        name = 'hidden'+str(i)
        num_neurons = randint(1, max_neurons_per_layer)
        activation = acti_func[randint(0, len(acti_func)-1)]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    global model_name
    global model_desc

<ipython-input-6-0e67b9f64f4e>:18: SyntaxWarning: name 'model_name' is assigned to before global declaration
  global model_name
<ipython-input-6-0e67b9f64f4e>:19: SyntaxWarning: name 'model_desc' is assigned to before global declaration
  global model_desc


In [7]:
def lr_and_optimizer(model_name):
    learning_rate = random.uniform(min_learning_rate, max_learning_rate)
    #print(learning_rate)
    model_desc.update({'learning_rate' : learning_rate})
    optimizer = Adam(lr=learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def print_summary(model_name):
    print('Neural Net Model Summary: ')
    print(model_name.summary())

In [9]:
def train_and_test(model_name):
    model_name.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)
    results = model_name.evaluate(test_x, test_y)
    global results

<ipython-input-9-915b75c55b69>:4: SyntaxWarning: name 'results' is assigned to before global declaration
  global results


In [10]:
def individual(model_num):
    create_network(model_num)
    #print_summary(model_name)
    lr_and_optimizer(model_name)
    train_and_test(model_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [11]:
def population():
    population_desc = []
    for x in xrange(num_individuals):
        individual(model_num = x)
        population_desc.append(model_desc)
    print(population_desc)
    global population_desc

<ipython-input-11-662326261e74>:7: SyntaxWarning: name 'population_desc' is assigned to before global declaration
  global population_desc


In [12]:
population()

Model0
Epoch 1/200
 - 1s - loss: 1.1513 - acc: 0.3000
Epoch 2/200
 - 0s - loss: 0.8000 - acc: 0.5917
Epoch 3/200
 - 0s - loss: 0.5145 - acc: 0.6833
Epoch 4/200
 - 0s - loss: 0.4730 - acc: 0.6917
Epoch 5/200
 - 0s - loss: 0.4775 - acc: 0.6833
Epoch 6/200
 - 0s - loss: 0.4687 - acc: 0.6667
Epoch 7/200
 - 0s - loss: 0.4929 - acc: 0.6333
Epoch 8/200
 - 0s - loss: 0.4628 - acc: 0.7083
Epoch 9/200
 - 0s - loss: 0.4670 - acc: 0.6833
Epoch 10/200
 - 0s - loss: 0.4697 - acc: 0.6500
Epoch 11/200
 - 0s - loss: 0.4756 - acc: 0.6917
Epoch 12/200
 - 0s - loss: 0.4476 - acc: 0.7333
Epoch 13/200
 - 0s - loss: 0.4870 - acc: 0.6833
Epoch 14/200
 - 0s - loss: 0.4675 - acc: 0.6583
Epoch 15/200
 - 0s - loss: 0.4781 - acc: 0.6833
Epoch 16/200
 - 0s - loss: 0.5818 - acc: 0.6333
Epoch 17/200
 - 0s - loss: 0.6123 - acc: 0.6500
Epoch 18/200
 - 0s - loss: 0.4746 - acc: 0.7000
Epoch 19/200
 - 0s - loss: 0.4363 - acc: 0.7167
Epoch 20/200
 - 0s - loss: 0.4349 - acc: 0.7917
Epoch 21/200
 - 0s - loss: 0.4794 - acc: 0

 - 0s - loss: 0.1923 - acc: 0.9417
Epoch 171/200
 - 0s - loss: 0.1628 - acc: 0.9167
Epoch 172/200
 - 0s - loss: 0.1098 - acc: 0.9750
Epoch 173/200
 - 0s - loss: 0.0547 - acc: 0.9833
Epoch 174/200
 - 0s - loss: 0.0527 - acc: 0.9833
Epoch 175/200
 - 0s - loss: 0.0485 - acc: 0.9833
Epoch 176/200
 - 0s - loss: 0.0460 - acc: 0.9833
Epoch 177/200
 - 0s - loss: 0.0564 - acc: 0.9833
Epoch 178/200
 - 0s - loss: 0.0641 - acc: 0.9833
Epoch 179/200
 - 0s - loss: 0.1186 - acc: 0.9583
Epoch 180/200
 - 0s - loss: 0.1009 - acc: 0.9583
Epoch 181/200
 - 0s - loss: 0.0634 - acc: 0.9833
Epoch 182/200
 - 0s - loss: 0.0710 - acc: 0.9750
Epoch 183/200
 - 0s - loss: 0.0681 - acc: 0.9750
Epoch 184/200
 - 0s - loss: 0.0570 - acc: 0.9667
Epoch 185/200
 - 0s - loss: 0.0540 - acc: 0.9833
Epoch 186/200
 - 0s - loss: 0.1195 - acc: 0.9583
Epoch 187/200
 - 0s - loss: 0.0970 - acc: 0.9417
Epoch 188/200
 - 0s - loss: 0.0541 - acc: 0.9833
Epoch 189/200
 - 0s - loss: 0.2389 - acc: 0.9167
Epoch 190/200
 - 0s - loss: 0.1239

 - 0s - loss: 0.4548 - acc: 0.7083
Epoch 131/200
 - 0s - loss: 0.4546 - acc: 0.7083
Epoch 132/200
 - 0s - loss: 0.4579 - acc: 0.7083
Epoch 133/200
 - 0s - loss: 0.4545 - acc: 0.7083
Epoch 134/200
 - 0s - loss: 0.4582 - acc: 0.7083
Epoch 135/200
 - 0s - loss: 0.4546 - acc: 0.7083
Epoch 136/200
 - 0s - loss: 0.4558 - acc: 0.7083
Epoch 137/200
 - 0s - loss: 0.4555 - acc: 0.7083
Epoch 138/200
 - 0s - loss: 0.4529 - acc: 0.7083
Epoch 139/200
 - 0s - loss: 0.4548 - acc: 0.7083
Epoch 140/200
 - 0s - loss: 0.4540 - acc: 0.7083
Epoch 141/200
 - 0s - loss: 0.4538 - acc: 0.7083
Epoch 142/200
 - 0s - loss: 0.4548 - acc: 0.7083
Epoch 143/200
 - 0s - loss: 0.4549 - acc: 0.7083
Epoch 144/200
 - 0s - loss: 0.4539 - acc: 0.7083
Epoch 145/200
 - 0s - loss: 0.4547 - acc: 0.7083
Epoch 146/200
 - 0s - loss: 0.4538 - acc: 0.7083
Epoch 147/200
 - 0s - loss: 0.4533 - acc: 0.7083
Epoch 148/200
 - 0s - loss: 0.4553 - acc: 0.7083
Epoch 149/200
 - 0s - loss: 0.4550 - acc: 0.7083
Epoch 150/200
 - 0s - loss: 0.4534

Epoch 90/200
 - 0s - loss: 0.0659 - acc: 0.9750
Epoch 91/200
 - 0s - loss: 0.0780 - acc: 0.9500
Epoch 92/200
 - 0s - loss: 0.0685 - acc: 0.9750
Epoch 93/200
 - 0s - loss: 0.0498 - acc: 0.9833
Epoch 94/200
 - 0s - loss: 0.0741 - acc: 0.9667
Epoch 95/200
 - 0s - loss: 0.1052 - acc: 0.9417
Epoch 96/200
 - 0s - loss: 0.0867 - acc: 0.9833
Epoch 97/200
 - 0s - loss: 0.0703 - acc: 0.9667
Epoch 98/200
 - 0s - loss: 0.0872 - acc: 0.9750
Epoch 99/200
 - 0s - loss: 0.0815 - acc: 0.9500
Epoch 100/200
 - 0s - loss: 0.1008 - acc: 0.9500
Epoch 101/200
 - 0s - loss: 0.0740 - acc: 0.9667
Epoch 102/200
 - 0s - loss: 0.0774 - acc: 0.9667
Epoch 103/200
 - 0s - loss: 0.0594 - acc: 0.9667
Epoch 104/200
 - 0s - loss: 0.0786 - acc: 0.9750
Epoch 105/200
 - 0s - loss: 0.0527 - acc: 0.9833
Epoch 106/200
 - 0s - loss: 0.0718 - acc: 0.9833
Epoch 107/200
 - 0s - loss: 0.0444 - acc: 0.9833
Epoch 108/200
 - 0s - loss: 0.1242 - acc: 0.9500
Epoch 109/200
 - 0s - loss: 0.0812 - acc: 0.9833
Epoch 110/200
 - 0s - loss: 0.

Epoch 51/200
 - 0s - loss: 0.4680 - acc: 0.7083
Epoch 52/200
 - 0s - loss: 0.4583 - acc: 0.7083
Epoch 53/200
 - 0s - loss: 0.4573 - acc: 0.7083
Epoch 54/200
 - 0s - loss: 0.4620 - acc: 0.6167
Epoch 55/200
 - 0s - loss: 0.4631 - acc: 0.7083
Epoch 56/200
 - 0s - loss: 0.4563 - acc: 0.6833
Epoch 57/200
 - 0s - loss: 0.4674 - acc: 0.7083
Epoch 58/200
 - 0s - loss: 0.4650 - acc: 0.7083
Epoch 59/200
 - 0s - loss: 0.4650 - acc: 0.7083
Epoch 60/200
 - 0s - loss: 0.4752 - acc: 0.5833
Epoch 61/200
 - 0s - loss: 0.4573 - acc: 0.7083
Epoch 62/200
 - 0s - loss: 0.4601 - acc: 0.6500
Epoch 63/200
 - 0s - loss: 0.4604 - acc: 0.7083
Epoch 64/200
 - 0s - loss: 0.4566 - acc: 0.7083
Epoch 65/200
 - 0s - loss: 0.4745 - acc: 0.7083
Epoch 66/200
 - 0s - loss: 0.4629 - acc: 0.6667
Epoch 67/200
 - 0s - loss: 0.4641 - acc: 0.7083
Epoch 68/200
 - 0s - loss: 0.4641 - acc: 0.6083
Epoch 69/200
 - 0s - loss: 0.4608 - acc: 0.7083
Epoch 70/200
 - 0s - loss: 0.4550 - acc: 0.7083
Epoch 71/200
 - 0s - loss: 0.4564 - acc:

...

{'hidden3': (37, 'hard_sigmoid'), 'hidden2': (6, 'hard_sigmoid'), 'hidden1': (19, 'softplus'), 'hidden0': (27, 'softmax'), 'optimizer': 'Adam', 'name': 'Model59', 'learning_rate': 0.009442213546025868, 'loss_fitness': 18.791583557794493, 'total_fitness': 23.791583557794493, 'accuracy_fitness': 5.0}
Model60
Epoch 1/200
 - 2s - loss: 1.9788 - acc: 0.3000
Epoch 2/200
 - 0s - loss: 1.1011 - acc: 0.4667
Epoch 3/200
 - 0s - loss: 0.8442 - acc: 0.5667
Epoch 4/200
 - 0s - loss: 0.7574 - acc: 0.6833
Epoch 5/200
 - 0s - loss: 0.5698 - acc: 0.7333
Epoch 6/200
 - 0s - loss: 0.5926 - acc: 0.6833
Epoch 7/200
 - 0s - loss: 0.5007 - acc: 0.7667
Epoch 8/200
 - 0s - loss: 0.4757 - acc: 0.7667
Epoch 9/200
 - 0s - loss: 0.4361 - acc: 0.7917
Epoch 10/200
 - 0s - loss: 0.4177 - acc: 0.8000
Epoch 11/200
 - 0s - loss: 0.3931 - acc: 0.8500
Epoch 12/200
 - 0s - loss: 0.4314 - acc: 0.7583
Epoch 13/200
 - 0s - loss: 0.3519 - acc: 0.8500
Epoch 14/200
 - 0s - loss: 0.3543 - acc: 0.8667
Epoch 15/200
 - 0s - loss: 0.

Epoch 163/200
 - 0s - loss: 0.0570 - acc: 0.9833
Epoch 164/200
 - 0s - loss: 0.0696 - acc: 0.9833
Epoch 165/200
 - 0s - loss: 0.0610 - acc: 0.9750
Epoch 166/200
 - 0s - loss: 0.0600 - acc: 0.9833
Epoch 167/200
 - 0s - loss: 0.0641 - acc: 0.9750
Epoch 168/200
 - 0s - loss: 0.0646 - acc: 0.9750
Epoch 169/200
 - 0s - loss: 0.0771 - acc: 0.9667
Epoch 170/200
 - 0s - loss: 0.0697 - acc: 0.9667
Epoch 171/200
 - 0s - loss: 0.0610 - acc: 0.9750
Epoch 172/200
 - 0s - loss: 0.0730 - acc: 0.9833
Epoch 173/200
 - 0s - loss: 0.0595 - acc: 0.9750
Epoch 174/200
 - 0s - loss: 0.0541 - acc: 0.9750
Epoch 175/200
 - 0s - loss: 0.0603 - acc: 0.9833
Epoch 176/200
 - 0s - loss: 0.0591 - acc: 0.9667
Epoch 177/200
 - 0s - loss: 0.0573 - acc: 0.9667
Epoch 178/200
 - 0s - loss: 0.0598 - acc: 0.9833
Epoch 179/200
 - 0s - loss: 0.0575 - acc: 0.9667
Epoch 180/200
 - 0s - loss: 0.0658 - acc: 0.9583
Epoch 181/200
 - 0s - loss: 0.0658 - acc: 0.9583
Epoch 182/200
 - 0s - loss: 0.0620 - acc: 0.9750
Epoch 183/200
 - 0s 

Epoch 127/200
 - 0s - loss: 0.0550 - acc: 0.9833
Epoch 128/200
 - 0s - loss: 0.0604 - acc: 0.9833
Epoch 129/200
 - 0s - loss: 0.0745 - acc: 0.9833
Epoch 130/200
 - 0s - loss: 0.0511 - acc: 0.9750
Epoch 131/200
 - 0s - loss: 0.1432 - acc: 0.9500
Epoch 132/200
 - 0s - loss: 0.1031 - acc: 0.9583
Epoch 133/200
 - 0s - loss: 0.0809 - acc: 0.9583
Epoch 134/200
 - 0s - loss: 0.0708 - acc: 0.9750
Epoch 135/200
 - 0s - loss: 0.1104 - acc: 0.9500
Epoch 136/200
 - 0s - loss: 0.0938 - acc: 0.9667
Epoch 137/200
 - 0s - loss: 0.0570 - acc: 0.9833
Epoch 138/200
 - 0s - loss: 0.0787 - acc: 0.9667
Epoch 139/200
 - 0s - loss: 0.0514 - acc: 0.9917
Epoch 140/200
 - 0s - loss: 0.0483 - acc: 0.9833
Epoch 141/200
 - 0s - loss: 0.0642 - acc: 0.9833
Epoch 142/200
 - 0s - loss: 0.0718 - acc: 0.9750
Epoch 143/200
 - 0s - loss: 0.0544 - acc: 0.9833
Epoch 144/200
 - 0s - loss: 0.0560 - acc: 0.9750
Epoch 145/200
 - 0s - loss: 0.0986 - acc: 0.9500
Epoch 146/200
 - 0s - loss: 0.0576 - acc: 0.9833
Epoch 147/200
 - 0s 

Epoch 87/200
 - 0s - loss: 0.0801 - acc: 0.9750
Epoch 88/200
 - 0s - loss: 0.0892 - acc: 0.9583
Epoch 89/200
 - 0s - loss: 0.0865 - acc: 0.9583
Epoch 90/200
 - 0s - loss: 0.0877 - acc: 0.9833
Epoch 91/200
 - 0s - loss: 0.0858 - acc: 0.9750
Epoch 92/200
 - 0s - loss: 0.0915 - acc: 0.9583
Epoch 93/200
 - 0s - loss: 0.0932 - acc: 0.9500
Epoch 94/200
 - 0s - loss: 0.0837 - acc: 0.9583
Epoch 95/200
 - 0s - loss: 0.0886 - acc: 0.9667
Epoch 96/200
 - 0s - loss: 0.0829 - acc: 0.9667
Epoch 97/200
 - 0s - loss: 0.0787 - acc: 0.9917
Epoch 98/200
 - 0s - loss: 0.0784 - acc: 0.9833
Epoch 99/200
 - 0s - loss: 0.0876 - acc: 0.9667
Epoch 100/200
 - 0s - loss: 0.0739 - acc: 0.9833
Epoch 101/200
 - 0s - loss: 0.0853 - acc: 0.9750
Epoch 102/200
 - 0s - loss: 0.0776 - acc: 0.9750
Epoch 103/200
 - 0s - loss: 0.0778 - acc: 0.9750
Epoch 104/200
 - 0s - loss: 0.0764 - acc: 0.9833
Epoch 105/200
 - 0s - loss: 0.0843 - acc: 0.9583
Epoch 106/200
 - 0s - loss: 0.0858 - acc: 0.9750
Epoch 107/200
 - 0s - loss: 0.074

 - 0s - loss: 0.0766 - acc: 0.9667
Epoch 50/200
 - 0s - loss: 0.0764 - acc: 0.9750
Epoch 51/200
 - 0s - loss: 0.1036 - acc: 0.9583
Epoch 52/200
 - 0s - loss: 0.0899 - acc: 0.9583
Epoch 53/200
 - 0s - loss: 0.0731 - acc: 0.9667
Epoch 54/200
 - 0s - loss: 0.0752 - acc: 0.9750
Epoch 55/200
 - 0s - loss: 0.0909 - acc: 0.9667
Epoch 56/200
 - 0s - loss: 0.0864 - acc: 0.9667
Epoch 57/200
 - 0s - loss: 0.0642 - acc: 0.9750
Epoch 58/200
 - 0s - loss: 0.1016 - acc: 0.9583
Epoch 59/200
 - 0s - loss: 0.0793 - acc: 0.9833
Epoch 60/200
 - 0s - loss: 0.0698 - acc: 0.9750
Epoch 61/200
 - 0s - loss: 0.0671 - acc: 0.9750
Epoch 62/200
 - 0s - loss: 0.0765 - acc: 0.9667
Epoch 63/200
 - 0s - loss: 0.0834 - acc: 0.9583
Epoch 64/200
 - 0s - loss: 0.0878 - acc: 0.9583
Epoch 65/200
 - 0s - loss: 0.0753 - acc: 0.9833
Epoch 66/200
 - 0s - loss: 0.0706 - acc: 0.9667
Epoch 67/200
 - 0s - loss: 0.0808 - acc: 0.9667
Epoch 68/200
 - 0s - loss: 0.0785 - acc: 0.9833
Epoch 69/200
 - 0s - loss: 0.0677 - acc: 0.9750
Epoch

In [13]:
def grade():    
    fitness_sum = 0
    'Find average fitness for a population.'
    for x in population_desc:
        fitness_sum = x.get("total_fitness") + fitness_sum
    avg_fitness = fitness_sum/len(population_desc)
    print(avg_fitness)
    global avg_fitness

<ipython-input-13-e0a3493f5de1>:8: SyntaxWarning: name 'avg_fitness' is assigned to before global declaration
  global avg_fitness


In [14]:
grade()

192.868786188


In [15]:
parents = []
for x in population_desc:
    if x.get("total_fitness") >= avg_fitness:
        parents.append((x.get('name'), x.get('total_fitness')))
        difference = x.get('total_fitness') - avg_fitness
        #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
def get_key(item):
    return item[1]
parents_sorted = sorted(parents, key = get_key, reverse = True)

if len(parents) % 2 !=0:
    parents_sorted.pop()
    print(parents_sorted)
else:
    print(parents_sorted)

[('Model47', 1225.261312881551), ('Model58', 1174.899648081001), ('Model46', 1021.9194167680871), ('Model60', 817.466563109623), ('Model26', 792.2354481425491), ('Model34', 699.9074614212697), ('Model48', 555.703948970003), ('Model25', 444.0804454720062), ('Model63', 431.4233042209216), ('Model21', 393.1092761211513), ('Model20', 363.65466838688184), ('Model53', 345.6097848836639), ('Model22', 345.3205982883295), ('Model0', 265.12316938635075), ('Model50', 253.14041460532923), ('Model7', 224.52632849348203), ('Model61', 200.4139423634344), ('Model57', 197.529552713655), ('Model62', 196.98139549001237), ('Model41', 193.94671364419656)]


In [16]:
def create_child(child_num, model_1, model_2):
    parent1 = (item for item in population_desc if item["name"] == model_1).next()
    parent2 = (item for item in population_desc if item["name"] == model_2).next()
    print(parent1)
    print(parent2)
    new_learning_rate = (parent1.get("learning_rate")+parent2.get("learning_rate"))/2
    print(new_learning_rate)
    def parent_select():
        num = randint(0,1)
        if num == 0:
            return parent1
        else:
            return parent2
    child_name = "Child"+str(child_num)
    child_desc = {'name':model_name}
    print(model_name)
    child_name = Sequential()
    child_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    avg_hidden = 0
    parent1_hidden = 0
    parent2_hidden = 0
    for key in parent1.keys():
        if "hidden" in key:
            parent1_hidden = parent1_hidden + 1
    for key in parent2.keys():
        if "hidden" in key:
            parent2_hidden = parent2_hidden + 1
    avg_hidden = (parent1_hidden + parent2_hidden)/2
    print(avg_hidden)
    i = 0
    while i < avg_hidden:
        name = 'hidden'+str(i)
        try:
            num_neurons = parent_select().get(name)[0]
            activation = parent_select().get(name)[1]
        except:
            try:
                num_neurons = parent1.get(name)[0]
                activation = parent1.get(name)[1]
            except:
                num_neurons = parent2.get(name)[0]
                activation = parent2.get(name)[1]
        child_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        child_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    child_name.add(Dense(3, activation='softmax', name='output'))
    child_desc.update({'learning_rate' : new_learning_rate})
    optimizer = Adam(lr=new_learning_rate)
    child_desc.update({'optimizer' : 'Adam'})
    child_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    global child_name
    global child_desc

<ipython-input-16-cf2cf0c206c4>:52: SyntaxWarning: name 'child_name' is assigned to before global declaration
  global child_name
<ipython-input-16-cf2cf0c206c4>:53: SyntaxWarning: name 'child_desc' is assigned to before global declaration
  global child_desc


In [17]:
def child(child_num, model_1, model_2):
    create_child(child_num, model_1, model_2)
    print_summary(child_name)
    lr_and_optimizer(child_name)
    train_and_test(child_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    child_desc.update({'loss_fitness':loss_fitness})
    child_desc.update({'accuracy_fitness':acc_fitness})
    child_desc.update({'total_fitness':total_fitness})
    print(child_name)
    print(child_desc)

In [18]:
def child_populations():
    child_population_desc = []
    for x in xrange(len(parents)):
        try:
            child(x, parents_sorted[x][0], parents_sorted[(x*2)+1][0])
            child_population_desc.append(child_desc)
        except:
            pass
    print(child_population_desc)
    global child_population_desc

<ipython-input-18-2bfd5361074b>:10: SyntaxWarning: name 'child_population_desc' is assigned to before global declaration
  global child_population_desc


In [19]:
child_populations()

{'hidden3': (78, 'softplus'), 'hidden2': (97, 'softplus'), 'hidden1': (95, 'softplus'), 'hidden0': (4, 'relu'), 'optimizer': 'Adam', 'name': 'Model47', 'learning_rate': 0.0008664474938127492, 'loss_fitness': 1215.261312881551, 'total_fitness': 1225.261312881551, 'accuracy_fitness': 10.0}
{'hidden2': (13, 'elu'), 'hidden1': (20, 'tanh'), 'hidden0': (82, 'sigmoid'), 'optimizer': 'Adam', 'name': 'Model58', 'learning_rate': 0.0032352402069491747, 'loss_fitness': 1164.899648081001, 'total_fitness': 1174.899648081001, 'accuracy_fitness': 10.0}
0.00205084385038
3
Neural Net Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 10)                50        
_________________________________________________________________
hidden0 (Dense)              (None, 82)                902       
_________________________________________________________________
hidden1 (Dense

Epoch 137/200
 - 0s - loss: 0.0538 - acc: 0.9667
Epoch 138/200
 - 0s - loss: 0.0635 - acc: 0.9667
Epoch 139/200
 - 0s - loss: 0.0601 - acc: 0.9667
Epoch 140/200
 - 0s - loss: 0.0425 - acc: 0.9833
Epoch 141/200
 - 0s - loss: 0.1410 - acc: 0.9500
Epoch 142/200
 - 0s - loss: 0.0592 - acc: 0.9750
Epoch 143/200
 - 0s - loss: 0.0616 - acc: 0.9750
Epoch 144/200
 - 0s - loss: 0.0764 - acc: 0.9667
Epoch 145/200
 - 0s - loss: 0.1019 - acc: 0.9500
Epoch 146/200
 - 0s - loss: 0.0983 - acc: 0.9500
Epoch 147/200
 - 0s - loss: 0.0562 - acc: 0.9833
Epoch 148/200
 - 0s - loss: 0.0461 - acc: 0.9750
Epoch 149/200
 - 0s - loss: 0.0613 - acc: 0.9833
Epoch 150/200
 - 0s - loss: 0.0465 - acc: 0.9833
Epoch 151/200
 - 0s - loss: 0.1221 - acc: 0.9667
Epoch 152/200
 - 0s - loss: 0.0763 - acc: 0.9667
Epoch 153/200
 - 0s - loss: 0.0589 - acc: 0.9750
Epoch 154/200
 - 0s - loss: 0.0669 - acc: 0.9583
Epoch 155/200
 - 0s - loss: 0.1764 - acc: 0.9250
Epoch 156/200
 - 0s - loss: 0.0713 - acc: 0.9667
Epoch 157/200
 - 0s 

Epoch 71/200
 - 0s - loss: 0.0630 - acc: 0.9750
Epoch 72/200
 - 0s - loss: 0.0722 - acc: 0.9500
Epoch 73/200
 - 0s - loss: 0.1197 - acc: 0.9417
Epoch 74/200
 - 0s - loss: 0.0696 - acc: 0.9833
Epoch 75/200
 - 0s - loss: 0.0682 - acc: 0.9750
Epoch 76/200
 - 0s - loss: 0.0671 - acc: 0.9583
Epoch 77/200
 - 0s - loss: 0.1178 - acc: 0.9500
Epoch 78/200
 - 0s - loss: 0.0799 - acc: 0.9667
Epoch 79/200
 - 0s - loss: 0.0492 - acc: 0.9750
Epoch 80/200
 - 0s - loss: 0.0598 - acc: 0.9833
Epoch 81/200
 - 0s - loss: 0.1142 - acc: 0.9500
Epoch 82/200
 - 0s - loss: 0.1072 - acc: 0.9583
Epoch 83/200
 - 0s - loss: 0.0532 - acc: 0.9667
Epoch 84/200
 - 0s - loss: 0.0780 - acc: 0.9667
Epoch 85/200
 - 0s - loss: 0.0801 - acc: 0.9667
Epoch 86/200
 - 0s - loss: 0.0606 - acc: 0.9833
Epoch 87/200
 - 0s - loss: 0.0842 - acc: 0.9583
Epoch 88/200
 - 0s - loss: 0.0526 - acc: 0.9750
Epoch 89/200
 - 0s - loss: 0.0589 - acc: 0.9833
Epoch 90/200
 - 0s - loss: 0.0778 - acc: 0.9583
Epoch 91/200
 - 0s - loss: 0.0560 - acc:

 - 2s - loss: 0.7624 - acc: 0.5917
Epoch 2/200
 - 0s - loss: 0.3898 - acc: 0.7667
Epoch 3/200
 - 0s - loss: 0.3657 - acc: 0.8000
Epoch 4/200
 - 0s - loss: 0.3665 - acc: 0.8250
Epoch 5/200
 - 0s - loss: 0.2381 - acc: 0.9000
Epoch 6/200
 - 0s - loss: 0.1496 - acc: 0.9583
Epoch 7/200
 - 0s - loss: 0.3393 - acc: 0.8667
Epoch 8/200
 - 0s - loss: 0.4874 - acc: 0.7417
Epoch 9/200
 - 0s - loss: 0.3104 - acc: 0.8583
Epoch 10/200
 - 0s - loss: 0.1732 - acc: 0.9167
Epoch 11/200
 - 0s - loss: 0.1108 - acc: 0.9750
Epoch 12/200
 - 0s - loss: 0.1031 - acc: 0.9750
Epoch 13/200
 - 0s - loss: 0.1276 - acc: 0.9500
Epoch 14/200
 - 0s - loss: 0.3617 - acc: 0.8583
Epoch 15/200
 - 0s - loss: 0.2274 - acc: 0.9000
Epoch 16/200
 - 0s - loss: 0.1448 - acc: 0.9417
Epoch 17/200
 - 0s - loss: 0.0960 - acc: 0.9667
Epoch 18/200
 - 0s - loss: 0.0866 - acc: 0.9583
Epoch 19/200
 - 0s - loss: 0.1039 - acc: 0.9667
Epoch 20/200
 - 0s - loss: 0.0765 - acc: 0.9667
Epoch 21/200
 - 0s - loss: 0.1141 - acc: 0.9500
Epoch 22/200


Epoch 171/200
 - 0s - loss: 0.0602 - acc: 0.9833
Epoch 172/200
 - 0s - loss: 0.0719 - acc: 0.9833
Epoch 173/200
 - 0s - loss: 0.0817 - acc: 0.9750
Epoch 174/200
 - 0s - loss: 0.0740 - acc: 0.9667
Epoch 175/200
 - 0s - loss: 0.0823 - acc: 0.9667
Epoch 176/200
 - 0s - loss: 0.0599 - acc: 0.9833
Epoch 177/200
 - 0s - loss: 0.0689 - acc: 0.9667
Epoch 178/200
 - 0s - loss: 0.0742 - acc: 0.9750
Epoch 179/200
 - 0s - loss: 0.0517 - acc: 0.9667
Epoch 180/200
 - 0s - loss: 0.1210 - acc: 0.9500
Epoch 181/200
 - 0s - loss: 0.0574 - acc: 0.9750
Epoch 182/200
 - 0s - loss: 0.0527 - acc: 0.9833
Epoch 183/200
 - 0s - loss: 0.1261 - acc: 0.9500
Epoch 184/200
 - 0s - loss: 0.1129 - acc: 0.9417
Epoch 185/200
 - 0s - loss: 0.0713 - acc: 0.9667
Epoch 186/200
 - 0s - loss: 0.0572 - acc: 0.9750
Epoch 187/200
 - 0s - loss: 0.0696 - acc: 0.9750
Epoch 188/200
 - 0s - loss: 0.0468 - acc: 0.9750
Epoch 189/200
 - 0s - loss: 0.0755 - acc: 0.9583
Epoch 190/200
 - 0s - loss: 0.1204 - acc: 0.9583
Epoch 191/200
 - 0s 

...

 - 0s - loss: 0.1363 - acc: 0.9583
Epoch 26/200
 - 0s - loss: 0.1327 - acc: 0.9417
Epoch 27/200
 - 0s - loss: 0.1182 - acc: 0.9583
Epoch 28/200
 - 0s - loss: 0.0926 - acc: 0.9667
Epoch 29/200
 - 0s - loss: 0.0763 - acc: 0.9667
Epoch 30/200
 - 0s - loss: 0.1144 - acc: 0.9667
Epoch 31/200
 - 0s - loss: 0.1365 - acc: 0.9500
Epoch 32/200
 - 0s - loss: 0.0801 - acc: 0.9667
Epoch 33/200
 - 0s - loss: 0.1052 - acc: 0.9667
Epoch 34/200
 - 0s - loss: 0.1199 - acc: 0.9500
Epoch 35/200
 - 0s - loss: 0.1360 - acc: 0.9500
Epoch 36/200
 - 0s - loss: 0.1134 - acc: 0.9583
Epoch 37/200
 - 0s - loss: 0.1659 - acc: 0.9167
Epoch 38/200
 - 0s - loss: 0.0759 - acc: 0.9750
Epoch 39/200
 - 0s - loss: 0.1386 - acc: 0.9250
Epoch 40/200
 - 0s - loss: 0.1502 - acc: 0.9500
Epoch 41/200
 - 0s - loss: 0.1246 - acc: 0.9583
Epoch 42/200
 - 0s - loss: 0.1675 - acc: 0.9500
Epoch 43/200
 - 0s - loss: 0.3176 - acc: 0.9167
Epoch 44/200
 - 0s - loss: 0.0986 - acc: 0.9667
Epoch 45/200
 - 0s - loss: 0.1043 - acc: 0.9583
Epoch

Epoch 194/200
 - 0s - loss: 0.0555 - acc: 0.9583
Epoch 195/200
 - 0s - loss: 0.1340 - acc: 0.9250
Epoch 196/200
 - 0s - loss: 0.1701 - acc: 0.8833
Epoch 197/200
 - 0s - loss: 0.1202 - acc: 0.9250
Epoch 198/200
 - 0s - loss: 0.0760 - acc: 0.9750
Epoch 199/200
 - 0s - loss: 0.0915 - acc: 0.9667
Epoch 200/200
 - 0s - loss: 0.0603 - acc: 0.9750
30/30 [==============================] - 1s 36ms/step
{'hidden3': (85, 'tanh'), 'hidden2': (31, 'softplus'), 'hidden1': (72, 'softsign'), 'hidden0': (86, 'softplus'), 'optimizer': 'Adam', 'name': <keras.engine.sequential.Sequential object at 0x7fd87347fed0>, 'learning_rate': 0.004209359749032306, 'loss_fitness': 395.804460675592, 'total_fitness': 405.804460675592, 'accuracy_fitness': 10.0}
{'hidden3': (5, 'linear'), 'hidden2': (78, 'elu'), 'hidden1': (97, 'elu'), 'hidden0': (16, 'softmax'), 'optimizer': 'Adam', 'name': 'Model25', 'hidden4': (100, 'linear'), 'learning_rate': 0.005581244346238733, 'loss_fitness': 434.0804454720062, 'accuracy_fitness':

Epoch 121/200
 - 0s - loss: 0.0662 - acc: 0.9750
Epoch 122/200
 - 0s - loss: 0.0629 - acc: 0.9750
Epoch 123/200
 - 0s - loss: 0.0701 - acc: 0.9750
Epoch 124/200
 - 0s - loss: 0.0770 - acc: 0.9667
Epoch 125/200
 - 0s - loss: 0.0848 - acc: 0.9667
Epoch 126/200
 - 0s - loss: 0.0561 - acc: 0.9750
Epoch 127/200
 - 0s - loss: 0.0699 - acc: 0.9750
Epoch 128/200
 - 0s - loss: 0.0620 - acc: 0.9667
Epoch 129/200
 - 0s - loss: 0.0837 - acc: 0.9583
Epoch 130/200
 - 0s - loss: 0.0969 - acc: 0.9500
Epoch 131/200
 - 0s - loss: 0.0724 - acc: 0.9583
Epoch 132/200
 - 0s - loss: 0.1209 - acc: 0.9500
Epoch 133/200
 - 0s - loss: 0.0511 - acc: 0.9667
Epoch 134/200
 - 0s - loss: 0.0576 - acc: 0.9750
Epoch 135/200
 - 0s - loss: 0.3154 - acc: 0.8917
Epoch 136/200
 - 0s - loss: 0.1842 - acc: 0.9417
Epoch 137/200
 - 0s - loss: 0.1304 - acc: 0.9250
Epoch 138/200
 - 0s - loss: 0.0826 - acc: 0.9667
Epoch 139/200
 - 0s - loss: 0.1253 - acc: 0.9500
Epoch 140/200
 - 0s - loss: 0.0947 - acc: 0.9583
Epoch 141/200
 - 0s 

 - 0s - loss: 0.4827 - acc: 0.7083
Epoch 47/200
 - 0s - loss: 0.4819 - acc: 0.7083
Epoch 48/200
 - 0s - loss: 0.4809 - acc: 0.7083
Epoch 49/200
 - 0s - loss: 0.4801 - acc: 0.7083
Epoch 50/200
 - 0s - loss: 0.4802 - acc: 0.7083
Epoch 51/200
 - 0s - loss: 0.4781 - acc: 0.7083
Epoch 52/200
 - 0s - loss: 0.4775 - acc: 0.7083
Epoch 53/200
 - 0s - loss: 0.4774 - acc: 0.7083
Epoch 54/200
 - 0s - loss: 0.4764 - acc: 0.7083
Epoch 55/200
 - 0s - loss: 0.4764 - acc: 0.7083
Epoch 56/200
 - 0s - loss: 0.4746 - acc: 0.7083
Epoch 57/200
 - 0s - loss: 0.4744 - acc: 0.7083
Epoch 58/200
 - 0s - loss: 0.4737 - acc: 0.7083
Epoch 59/200
 - 0s - loss: 0.4730 - acc: 0.7083
Epoch 60/200
 - 0s - loss: 0.4727 - acc: 0.7083
Epoch 61/200
 - 0s - loss: 0.4719 - acc: 0.7083
Epoch 62/200
 - 0s - loss: 0.4719 - acc: 0.7083
Epoch 63/200
 - 0s - loss: 0.4708 - acc: 0.7083
Epoch 64/200
 - 0s - loss: 0.4706 - acc: 0.7083
Epoch 65/200
 - 0s - loss: 0.4700 - acc: 0.7083
Epoch 66/200
 - 0s - loss: 0.4700 - acc: 0.7083
Epoch

Epoch 1/200
 - 3s - loss: 1.3605 - acc: 0.3417
Epoch 2/200
 - 0s - loss: 1.0941 - acc: 0.3667
Epoch 3/200
 - 0s - loss: 0.7431 - acc: 0.6667
Epoch 4/200
 - 0s - loss: 0.5398 - acc: 0.6917
Epoch 5/200
 - 0s - loss: 0.5448 - acc: 0.6833
Epoch 6/200
 - 0s - loss: 0.5087 - acc: 0.6917
Epoch 7/200
 - 0s - loss: 0.4936 - acc: 0.6833
Epoch 8/200
 - 0s - loss: 0.4806 - acc: 0.7250
Epoch 9/200
 - 0s - loss: 0.4707 - acc: 0.6750
Epoch 10/200
 - 0s - loss: 0.4867 - acc: 0.6500
Epoch 11/200
 - 0s - loss: 0.5768 - acc: 0.6583
Epoch 12/200
 - 0s - loss: 0.5172 - acc: 0.6667
Epoch 13/200
 - 0s - loss: 0.5471 - acc: 0.6833
Epoch 14/200
 - 0s - loss: 0.5571 - acc: 0.6833
Epoch 15/200
 - 0s - loss: 0.4712 - acc: 0.6833
Epoch 16/200
 - 0s - loss: 0.4596 - acc: 0.6583
Epoch 17/200
 - 0s - loss: 0.5025 - acc: 0.6333
Epoch 18/200
 - 0s - loss: 0.4765 - acc: 0.7083
Epoch 19/200
 - 0s - loss: 0.4645 - acc: 0.7000
Epoch 20/200
 - 0s - loss: 0.4319 - acc: 0.7083
Epoch 21/200
 - 0s - loss: 0.3911 - acc: 0.7917
E

In [ ]:
parents = []
for x in child_population_desc:
    if x.get("total_fitness") >= avg_fitness:
        parents.append((x.get('name'), x.get('total_fitness')))
        difference = x.get('total_fitness') - avg_fitness
        #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
def get_key(item):
    return item[1]
parents_sorted = sorted(parents, key = get_key, reverse = True)

if len(parents) % 2 !=0:
    parents_sorted.pop()
    print(parents_sorted)
else:
    print(parents_sorted)